In [60]:
import sys
import profiler
import numpy as np
import six
import py_entitymatching as em
import pandas as pd
import os
import py_entitymatching.catalog.catalog_manager as cm
from py_entitymatching.utils.validation_helper import validate_object_type

In [61]:
print('python version: ' + sys.version )
print('pandas version: ' + pd.__version__ )
print('magellan version: ' + em.__version__ )

python version: 3.6.8 |Anaconda, Inc.| (default, Feb 21 2019, 18:30:04) [MSC v.1916 64 bit (AMD64)]
pandas version: 0.24.2
magellan version: 0.3.1


In [62]:
A = em.read_csv_metadata('ltable.csv', encoding='ISO-8859-1')
B= em.read_csv_metadata('rtable.csv',  encoding='ISO-8859-1')
em.set_key(A, 'ltable_id')
em.set_key(B, 'rtable_id')

Metadata file is not present in the given path; proceeding to read the csv file.
Metadata file is not present in the given path; proceeding to read the csv file.


True

In [63]:
S = em.read_csv_metadata("train.csv", 
                         key='id',
                         ltable=A, rtable=B, 
                         fk_ltable='ltable_id', fk_rtable='rtable_id')

Metadata file is not present in the given path; proceeding to read the csv file.


In [64]:
# Create a set of ML-matchers
# dt = em.DTMatcher(name='DecisionTree')
# svm = em.SVMMatcher(name='SVM')
# rf = em.RFMatcher(name='RF')
# nb = em.NBMatcher(name='NB')
lg = em.LogRegMatcher(name='LogReg')
# ln = em.LinRegMatcher(name='LinReg')

In [65]:
feature_table = em.get_features_for_matching(A, B, validate_inferred_attr_types=False)

In [29]:
# Extract feature vectors
feature_vectors_dev = em.extract_feature_vecs(S, 
                            feature_table=feature_table, 
                            attrs_after='label')

0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:11


In [30]:
# Impute feature vectors with the mean of the column values.
feature_vectors_dev = em.impute_table(feature_vectors_dev, 
                exclude_attrs=['id', 'ltable_id', 'rtable_id', 'label'],
                strategy='mean')

C:\Users\Chef\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:58: DeprecationWarning: Class Imputer is deprecated; Imputer was deprecated in version 0.20 and will be removed in 0.22. Import impute.SimpleImputer from sklearn instead.
  warnings.warn(msg, category=DeprecationWarning)


In [50]:
result = em.select_matcher([dt, rf, svm, ln, lg, nb], table=feature_vectors_dev, 
        exclude_attrs=['id', 'ltable_id', 'rtable_id', 'label'],
        k=5,
        target_attr='label', metric_to_select_matcher='recall', random_state=0)
result['cv_stats']

,Matcher,Average precision,Average recall,Average f1
0,DecisionTree,0.594650,0.567632,0.572257
1,RF,0.813848,0.491170,0.616241
2,SVM,0.904368,0.335341,0.489210
3,LinReg,0.972177,0.438797,0.603510
4,LogReg,0.943515,0.452186,0.609692
5,NB,0.478641,0.544630,0.507736


In [32]:
feature_vectors_dev.head()

,id,ltable_id,rtable_id,category_category_jac_qgm_3_qgm_3,category_category_cos_dlm_dc0_dlm_dc0,category_category_jac_dlm_dc0_dlm_dc0,category_category_mel,category_category_lev_dist,category_category_lev_sim,category_category_nmw,...,modelno_modelno_mel,modelno_modelno_lev_dist,modelno_modelno_lev_sim,modelno_modelno_nmw,modelno_modelno_sw,price_price_exm,price_price_anm,price_price_lev_dist,price_price_lev_sim,label
0,1,621,3167,0.054054,0.000000,0.000000,0.484127,16.0,0.238095,-2.0,...,0.457936,12.00000,0.142857,0.000000,2.000000,0.000000,0.741408,3.00000,0.500000,0
1,2,2115,8789,0.000000,0.000000,0.000000,0.577249,17.0,0.190476,1.0,...,0.658378,6.25228,0.426321,1.772905,4.083587,0.000000,0.412166,4.00000,0.333333,0
2,3,278,9064,0.023810,0.000000,0.000000,0.671958,15.0,0.285714,1.0,...,0.658378,6.25228,0.426321,1.772905,4.083587,0.023509,0.605909,3.53322,0.342763,0
3,4,1673,13279,0.578947,0.816497,0.666667,0.865000,6.0,0.625000,4.0,...,0.658378,6.25228,0.426321,1.772905,4.083587,0.000000,0.916528,1.00000,0.800000,0
4,5,2305,2948,0.058824,0.000000,0.000000,0.430880,16.0,0.238095,-6.0,...,0.447619,6.00000,0.142857,-1.000000,1.000000,0.000000,0.899354,3.00000,0.400000,0


In [36]:
path_L ='test.csv'
L = em.read_csv_metadata(path_L, 
                         key='id',
                         ltable=A, rtable=B, 
                         fk_ltable='ltable_id', fk_rtable='rtable_id')

Metadata file is not present in the given path; proceeding to read the csv file.


In [37]:
test_table = em.extract_feature_vecs(L, feature_table=feature_table, show_progress=False)

In [42]:
L = em.impute_table(test_table, 
                exclude_attrs=['id', 'ltable_id', 'rtable_id'],
                strategy='mean')


C:\Users\Chef\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:58: DeprecationWarning: Class Imputer is deprecated; Imputer was deprecated in version 0.20 and will be removed in 0.22. Import impute.SimpleImputer from sklearn instead.
  warnings.warn(msg, category=DeprecationWarning)


In [48]:
lg.fit(table=feature_vectors_dev, 
       exclude_attrs=['id', 'ltable_id', 'rtable_id', 'label'], 
       target_attr='label')

predictions = lg.predict(table=L, exclude_attrs=['id', 'ltable_id', 'rtable_id'], 
                             append=True, target_attr='label', inplace=False)

C:\Users\Chef\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [52]:
dataset = pd.DataFrame({"id":results[0]['id'],'label':predictions['label']})
dataset.to_csv("./prediction2.csv", index=False)

In [51]:
# Train using feature vectors from I 
classifiers = list()
classifiers.append(dt)
classifiers.append(svm)
classifiers.append(rf)
classifiers.append(nb)
classifiers.append(lg)
classifiers.append(ln)
results = list()
for i in classifiers:
    method = i
    method.fit(table=feature_vectors_dev, 
       exclude_attrs=['id', 'ltable_id', 'rtable_id', 'label'], 
       target_attr='label')
    predictions = method.predict(table=L, exclude_attrs=['id', 'ltable_id', 'rtable_id'], 
                             append=True, target_attr='label', inplace=False)
    print(predictions['label'][1:10])
    results.append(predictions)


1    0
2    1
3    0
4    0
5    0
6    0
7    0
8    1
9    0
Name: label, dtype: int64


C:\Users\Chef\Anaconda3\lib\site-packages\sklearn\svm\base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


1    0
2    1
3    0
4    0
5    0
6    0
7    0
8    0
9    0
Name: label, dtype: int64
1    0
2    1
3    0
4    0
5    0
6    0
7    0
8    0
9    0
Name: label, dtype: int64
1    0
2    1
3    0
4    0
5    0
6    0
7    0
8    0
9    0
Name: label, dtype: int64
1    0
2    1
3    0
4    0
5    0
6    0
7    0
8    0
9    0
Name: label, dtype: int64
1    0
2    1
3    0
4    0
5    0
6    0
7    0
8    0
9    0
Name: label, dtype: int32


C:\Users\Chef\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\Chef\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [53]:
# Predict on L 
first = results[0]

final = np.zeros(len(first['label']))
for item in results:
    final = final + item['label']
print(final)        

0       0.0
1       0.0
2       6.0
3       0.0
4       0.0
5       0.0
6       0.0
7       0.0
8       1.0
9       0.0
10      0.0
11      0.0
12      1.0
13      0.0
14      1.0
15      0.0
16      1.0
17      0.0
18      0.0
19      0.0
20      0.0
21      0.0
22      0.0
23      0.0
24      6.0
25      0.0
26      0.0
27      1.0
28      0.0
29      0.0
       ... 
3063    1.0
3064    0.0
3065    5.0
3066    0.0
3067    6.0
3068    0.0
3069    1.0
3070    1.0
3071    0.0
3072    0.0
3073    0.0
3074    0.0
3075    0.0
3076    1.0
3077    0.0
3078    0.0
3079    0.0
3080    0.0
3081    0.0
3082    1.0
3083    1.0
3084    0.0
3085    0.0
3086    1.0
3087    1.0
3088    0.0
3089    0.0
3090    0.0
3091    1.0
3092    0.0
Name: label, Length: 3093, dtype: float64


In [54]:
finalResult = final
for i in range(len(final)):
    print(final[i])
    if final[i] >= 3.0:
        finalResult[i] = 1
    else:
        finalResult[i] = 0


0.0
0.0
6.0
0.0
0.0
0.0
0.0
0.0
1.0
0.0
0.0
0.0
1.0
0.0
1.0
0.0
1.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
6.0
0.0
0.0
1.0
0.0
0.0
1.0
0.0
0.0
0.0
0.0
2.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
1.0
0.0
1.0
0.0
0.0
5.0
0.0
1.0
0.0
0.0
1.0
0.0
0.0
0.0
0.0
1.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
1.0
6.0
0.0
0.0
0.0
0.0
0.0
1.0
0.0
1.0
0.0
0.0
0.0
0.0
0.0
1.0
0.0
0.0
0.0
0.0
6.0
0.0
1.0
0.0
0.0
0.0
0.0
0.0
0.0
1.0
0.0
0.0
0.0
0.0
1.0
3.0
6.0
0.0
0.0
1.0
0.0
0.0
0.0
0.0
1.0
0.0
0.0
1.0
1.0
2.0
0.0
0.0
0.0
0.0
6.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
1.0
0.0
0.0
1.0
0.0
1.0
0.0
1.0
2.0
0.0
0.0
6.0
1.0
6.0
0.0
0.0
0.0
0.0
1.0
0.0
0.0
1.0
0.0
0.0
1.0
0.0
0.0
1.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
1.0
0.0
0.0
0.0
0.0
0.0
0.0
1.0
0.0
0.0
0.0
1.0
0.0
6.0
0.0
1.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
1.0
5.0
0.0
0.0
0.0
0.0
6.0
0.0
0.0
0.0
0.0
0.0
6.0
0.0
0.0
6.0
0.0
0.0
1.0
1.0
0.0
0.0
0.0
0.0
0.0
0.0
1.0
1.0
1.0
0.0
0.0
1.0
1.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
6.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0


0.0
0.0
0.0
1.0
0.0
0.0
0.0
0.0
1.0
0.0
1.0
0.0
1.0
6.0
0.0
0.0
0.0
0.0
0.0
0.0
1.0
0.0
1.0
0.0
1.0
1.0
0.0
6.0
0.0
0.0
1.0
0.0
0.0
0.0
0.0
0.0
1.0
0.0
1.0
0.0
0.0
0.0
0.0
1.0
3.0
0.0
1.0
0.0
1.0
0.0
0.0
0.0
1.0
0.0
0.0
0.0
0.0
5.0
0.0
0.0
1.0
0.0
0.0
0.0
0.0
0.0
1.0
1.0
1.0
0.0
0.0
0.0
6.0
0.0
1.0
0.0
0.0
0.0
0.0
0.0
1.0
0.0
0.0
0.0
0.0
1.0
0.0
0.0
0.0
0.0
1.0
0.0
0.0
0.0
1.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
1.0
0.0
0.0
0.0
6.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
1.0
0.0
0.0
0.0
0.0
0.0
0.0
2.0
4.0
1.0
0.0
0.0
0.0
2.0
2.0
0.0
0.0
0.0
1.0
0.0
1.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
1.0
0.0
0.0
0.0
0.0
0.0
0.0
1.0
0.0
0.0
0.0
0.0
0.0
0.0
2.0
0.0
0.0
0.0
6.0
0.0
1.0
1.0
0.0
0.0
0.0
0.0
0.0
0.0
1.0
0.0
0.0
0.0
0.0
1.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
6.0
0.0
0.0
0.0
0.0
0.0
6.0
0.0
0.0
6.0
1.0
0.0
0.0
2.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
1.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
1.0
1.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
1.0
0.0
0.0
0.0
0.0
0.0
0.0
1.0
0.0
0.0


In [55]:
dataset = pd.DataFrame({"id":results[0]['id'],'label':finalResult.astype(int)})
dataset.to_csv("./prediction2.csv", index=False)